In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np


def get_t(D):
    return 2 * (5 * D + 4) ** 0.5 - 4


def get_D(t):
    return ((t / 2 + 2) ** 2 - 4) / 5


ddD = -0.1
ddD2 = 0.2


def print_Dt(*tds):
    dD = -0.4
    t = 0
    D = 0
    left_t = 0
    for _t, _D in tds:
        if _t == 'next':
            _t = t + left_t
        assert _t >= t
        if left_t + t <= _t:
            # 这段时间dD增加
            t += left_t
            dD += ddD * left_t
            D = 0
            print(f"{t=:.2f} {D=:.3f} {dD=:.3f}")

            dD = min(-0.4, dD + ddD2 * (_t - t))
            D = _D
            t = _t
            old_D = 0
        else:
            # 可以延续
            dt = _t - t
            D += dt * (dD + ddD * dt / 2)
            t = _t
            dD += ddD * dt
            old_D = D
            D = max(_D, D)
        left_t = (dD + (dD * dD - 2 * D * ddD) ** 0.5) / -ddD
        print(f"{t=:.2f} D:{old_D:.3f}=>{D:.3f} {dD=:.3f} to={t+left_t:.2f}")


def print_Dt2(*tds):
    ddD = -0.1
    ddD2 = 0.2
    dD = -0.4
    t = 0
    D = 0
    left_t = 0
    res = []
    for _t, _D, *_msg in tds:
        if _t == 'next':
            _t = t + left_t
        assert _t >= t
        if left_t + t <= _t:
            # 这段时间dD增加
            t += left_t
            dD += ddD * left_t
            D = 0
            res.append([t, 0, dD, 0])
            dD = min(-0.4, dD + ddD2 * (_t - t))
            D = _D
            t = _t
            old_D = 0
            res.append([t, 0, dD, D])
        else:
            # 可以延续
            dt = _t - t
            D += dt * (dD + ddD * dt / 2)
            t = _t
            dD += ddD * dt
            D = max(_D, D)
            res.append([t, 0, dD, D])
        if _msg:
            res[-1].append(_msg[0])
        left_t = (dD + (dD * dD - 2 * D * ddD) ** 0.5) / -ddD
    for i in range(len(res)-1):
        res[i][1] = res[i+1][0]
    res.pop()
    return res


def print_rule(rules, title):
    ts = np.arange(0, rules[-1][1], 0.01)
    dDs = np.zeros_like(ts)
    Ds = np.zeros_like(ts)
    dDs[0] = -0.4
    Ds[0] = 0
    for t1, t2, dD, D, *_ in rules:
        idxs = np.where((ts > t1) & (ts <= t2))
        for idx in idxs:
            t = ts[idx]
            if D > 0:
                dDs[idx] = dD + ddD * (t - t1)
                Ds[idx] = D + (t - t1) * (dD + ddD * (t - t1) / 2)
            else:
                dDs[idx] = np.minimum(-0.4, dD + ddD2 * (t - t1))
                Ds[idx] = 0
    plt.figure(figsize=(5, 2.7), layout='constrained')
    plt.plot(ts, -dDs, label='dD/dt')  # Plot some data on the (implicit) axes.
    plt.plot(ts, Ds, label='D')  # etc.
    plt.plot(ts, np.zeros_like(ts), linestyle='--')  # etc.
    plt.xlabel('t')
    plt.ylabel('D')
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
    plt.title(title)
    for t1, t2, dD, D, *msg in rules:
        if not msg:
            continue
        plt.annotate(msg[0], xy=(t1, D), xytext=(t1+0.2, D+0.3),
                     arrowprops=dict(facecolor='black', width=0.1, headwidth=3, headlength=5))
    plt.ylim(-0.5, 2.5)
    plt.savefig(fname= title + ".png")
    plt.legend()

In [ ]:
rules = print_Dt2((0, 2), (2, 1.6), (4, 1.6), (6, 1.6), ('next', 0))  # 心海-丘丘人
print_rule(rules, "万叶E 心海 丘丘人")

In [ ]:
rules = print_Dt2((0, 2), (2, 2), (4, 2), (6, 2), ('next', 0))  # 心海-丘丘人
print_rule(rules, "万叶EQ 心海 丘丘人")

In [ ]:
rules = print_Dt2(
    (0, 2, 'Q1'), (0.5, 0, '放E'), (1.5, 1.6, 'E1'), ('next', 1.6, 'Q2'),
    (4.5, 1.6, 'E4'), (5.5, 1.6, 'E5'), ('next', 0))  # 莫娜QE-丘丘人
print_rule(rules, "万叶E 莫娜QE 丘丘人")

In [ ]:
rules = print_Dt2(
    (0, 0, '放E'), (1, 2, 'E1'), (4, 1.6, 'E4'), (5, 1.6, 'E5'), ('next', 0))
print_rule(rules, "万叶E 莫娜E 丘丘人")

In [ ]:
rules = print_Dt2(
    (0, 1, 'Q1'), (0.5, 0, '放E'), (1.5, 1, 'E1'), ('next', 2, 'Q2'),
    (4.5, 1, 'E4'), (5.5, 1, 'E5'), ('next', 0))
print_rule(rules, "万叶EQ 莫娜QE 圣骸兽")

In [ ]:
rules = print_Dt2((0, 1, 'shift'), (1, 1, 'Q1'), (2.5, 1, 'E1'), ('next', 2, 'Q2'),
                  (5.5, 1, 'E4'), (6.5, 1, 'E5'), ('next', 0))  # 莫娜-圣骸兽,先冲刺
print_rule(rules, "万叶EQ 莫娜冲刺QE 圣骸兽")

In [ ]:
rules = print_Dt2((0, 1), (2, 1), (4, 1), (6, 1), ('next', 0))  # 心海-圣骸兽
print_rule(rules, "万叶EQ 心海 圣骸兽")

莫娜E技能，持续5秒，释放瞬间不挂水，每秒攻击一次，2.5s/3hit，最后一次攻击独立附着，
因此放E后第1,4,5秒挂1水；
莫娜Q技能，释放瞬间挂1水，破裂时挂2单位强水，但是两次挂水共用2.5s计时器，
也就是Q后至少等2.5s破裂才会挂强水；
冻结时，莫娜Q不会破裂，非冻结情况下，随便一个带削韧的攻击都可以使泡泡破裂；

E的五次攻击，简称E1,...,E4,E5;
Q的五次攻击，简称Q1,Q2
由此，为了使Q2能挂强水，Q1后怪物保持冻结至少需要2.5s。
正常的怪甚至不需要E的参与

圣骸兽有50%冻结抗性，如1单位冰+1单位水=2单位冻，但是只能留下1单位冻元素；
冰冻共存，扩散反应优先扩散藏元素；
0.5冰+1风=2.2冰；
群怪条件，神里万叶都开大时，可以认为怪物头上始终附着有接近2单位冰元素；
